In [1]:
# pip install pandas
import pandas as pd
import numpy as np
import timeit
import os

In [2]:
# See how many files we have
number_of_files = 122472
number_of_files

122472

In [15]:
# See what the file looks like
df = pd.read_csv("test.csv", index_col=0)
df

,0,4096,-1046,0.1,-4,-648,-40,-17,28,-254,...,-10.26,-22.16,4.27,-20.15,19.13,-19.23,19.14,2.34,13.20,1.35
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:

# Convert the fft representation of the audio back to the waveform
def fft_to_audio(fft_df, fs = 44100):
    fft_df = np.array(fft_df.drop(fft_df.columns[[0,1]], axis=1))
    arr = np.zeros((fft_df.shape[0], 4096))
    arr[:fft_df.shape[0], :fft_df.shape[1]] = fft_df
    lst = []
    
    for ar in arr:
        i = 0
        lst1=[]
        while i < len(ar):
            lst1.append(complex(ar[i], ar[i+1]))
            i+=2
        lst.append(lst1)
        
    fft = np.array(lst)
    data = np.fft.irfft(fft)
    wav_data = np.concatenate(data)
    return wav_data

In [6]:
fft_to_audio(df)

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
time = timeit.timeit('fft_to_audio(df)', number=1, globals=globals())
time

2.200330125000619

In [17]:
print(str(number_of_files * time) + " seconds")
print(str((number_of_files * time)/60) + " minutes")
print(str((number_of_files * time)/3600) + " hours")
print(str((number_of_files * time)/86400) + " days")

269478.8310690758 seconds
4491.313851151263 minutes
74.85523085252106 hours
3.118967952188377 days


In [9]:
def fft_to_audio_optimized(fft_df, fs = 44100):
    fft_df = np.array(fft_df.drop(fft_df.columns[[0,1]], axis=1))
    arr = np.zeros((fft_df.shape[0], 4096))
    arr[:fft_df.shape[0], :fft_df.shape[1]] = fft_df

    fft = arr[::,::2] + (arr[::,1::2]*1j)
    
    data = np.fft.irfft(fft)
    wav_data = np.concatenate(data)
    return wav_data

In [18]:
fft_to_audio_optimized(df)

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
time = timeit.timeit('fft_to_audio_optimized(df)', number=1, globals=globals())
time

0.2219357969952398

In [12]:
print(str(number_of_files * time) + " seconds")
print(str((number_of_files * time)/60) + " minutes")
print(str((number_of_files * time)/3600) + " hours")
print(str((number_of_files * time)/86400) + " days")

24817.831895966723 seconds
413.63053159944536 minutes
6.893842193324089 hours
0.28724342472183706 days


In [21]:
if False:
    from joblib import Parallel, delayed
    res = Parallel(n_jobs=64)(delayed(fft_to_audio_optimized)(i) for i in all_files)# won't o






In [24]:
print(str(((number_of_files) * time)/64) + " seconds")
print(str((((number_of_files) * time)/64)/60) + " minutes")
print(str((((number_of_files) * time)/64)/3600) + " hours")
print(str((((number_of_files) * time)/64)/86400) + " days")

424.70188952501576 seconds
7.078364825416929 minutes
0.11797274709028216 hours
0.004915531128761756 days
